# Twitter Political Classification

The goal of this project is to determine US Political Party Affiliation by Twitter Usage

## Getting Data

In [1]:
import twitterscraper

INFO: {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201'}


"Democrats" selected are Bernie Sanders, Elizabeth Warren, and Joe Biden. These will be encoded as party=0

In [2]:
democratHandles = ["berniesanders","ewarren","JoeBiden"]

"Republicans" selected are Mitch McConnel, Ted Cruz, and Devin Nunes

In [3]:
republicanHandles = ["tedcruz", "senatemajldr", "DevinNunes"]

In [4]:
demTweets = [twitterscraper.query_tweets_from_user(x, 500) for x in democratHandles]

INFO: Scraping tweets from https://twitter.com/berniesanders
INFO: Using proxy 123.209.96.233:8080
INFO: Scraping tweets from https://twitter.com/i/profiles/show/berniesanders/timeline/tweets?include_available_features=1&include_entities=1&max_position=1220360497926692864&reset_error_state=false
INFO: Using proxy 181.196.242.126:53281
ERROR: An unknown error occurred! Returning tweets gathered so far.
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/twitterscraper/query.py", line 255, in query_tweets_from_user
    new_tweets, pos = query_single_page(user, lang='', pos=pos, from_user=True)
  File "/usr/local/lib/python2.7/dist-packages/twitterscraper/query.py", line 139, in query_single_page
    except json.decoder.JSONDecodeError as e:
AttributeError: 'module' object has no attribute 'JSONDecodeError'
INFO: Got 13 tweets from username berniesanders.
INFO: Scraping tweets from https://twitter.com/ewarren
INFO: Using proxy 212.73.69.6:59763
INFO: Scraping

In [5]:
demTweets = [x for s in demTweets for x in s]

In [6]:
demTweets = [{"text":x.text, "party":0} for x in demTweets]

In [7]:
demTweets[0]

{'party': 0,
 'text': u'Congratulations to labor activists and working class Minneapolitans on another hard fought victory to raise the minimum wage to $15 an hour. Now we must follow their lead and ensure a livable wage in all 50 states.http://www.startribune.com/minnesota-supreme-court-says-minneapolis-15-minimum-wage-can-stand/567197132/?om_rid=57792626121&om_mid=627378633\xa0\u2026'}

<span style="color:red">TODO -- This semeed to fail to get as many tweets as expected. Something wrong with scraper?</span>

In [8]:
repTweets = [twitterscraper.query_tweets_from_user(x, 500) for x in republicanHandles]

INFO: Scraping tweets from https://twitter.com/tedcruz
INFO: Using proxy 159.65.13.147:8080
INFO: Scraping tweets from https://twitter.com/i/profiles/show/tedcruz/timeline/tweets?include_available_features=1&include_entities=1&max_position=1220442990931369984&reset_error_state=false
INFO: Using proxy 193.200.151.158:57532
ERROR: An unknown error occurred! Returning tweets gathered so far.
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/twitterscraper/query.py", line 255, in query_tweets_from_user
    new_tweets, pos = query_single_page(user, lang='', pos=pos, from_user=True)
  File "/usr/local/lib/python2.7/dist-packages/twitterscraper/query.py", line 139, in query_single_page
    except json.decoder.JSONDecodeError as e:
AttributeError: 'module' object has no attribute 'JSONDecodeError'
INFO: Got 17 tweets from username tedcruz.
INFO: Scraping tweets from https://twitter.com/senatemajldr
INFO: Using proxy 193.193.240.37:45944
INFO: Scraping tweets fro

In [9]:
repTweets = [x for s in repTweets for x in s]

In [10]:
repTweets = [{"text":x.text, "party":1} for x in repTweets]

In [11]:
repTweets[0]

{'party': 1,
 'text': u'A new, right-off-the-senate-floor podcast on the #ImpeachmentTrial was launched by Sen. @tedcruz and commentator @michaeljknowles.https://thetexan.news/sen-ted-cruz-starts-impeachment-podcast-with-the-daily-wires-michael-knowles/\xa0\u2026'}

Ok, we have some labeled tweets now. Let's put them together.

In [12]:
labeledTweets = repTweets + demTweets

How many do we have?

In [13]:
len(labeledTweets)

88

88 (as of writing this). 88?! That's not very good. Despite better judgement, I'll try this anyway.

## The Model

Especially considring how bad this data is, let's be lazy and use keras

In [14]:
import tensorflow as tf
import numpy as np

In [15]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string], '')
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()


We don't have enough for good validation now, and we have very little data, so I'll just do 90% train 10% test

In [16]:
import random, math

In [17]:
random.shuffle(labeledTweets)

In [18]:
splitNum = int(math.floor(len(labeledTweets)*0.1))

In [19]:
testData = labeledTweets[:splitNum]

In [20]:
trainData = labeledTweets[splitNum:]

In [21]:
print("Training Examples: " + str(len(trainData)) + " Test Examples: " + str(len(testData)))

Training Examples: 80 Test Examples: 8


let's make this a dataset, then an input

In [22]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)

In [23]:
tokenizer.fit_on_texts([x['text'] for x in trainData])

In [24]:
tokenizer.fit_on_texts([x['text'] for x in testData])

In [25]:
X_train = tokenizer.texts_to_sequences([x['text'] for x in trainData])

In [26]:
X_test = tokenizer.texts_to_sequences([x['text'] for x in testData])

In [27]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding="post", maxlen=20)

In [28]:
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding="post", maxlen=20)

In [29]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(5000, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [30]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['binary_accuracy'])


In [31]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 32)                6272      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 166,817
Trainable params: 166,817
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
Y_train = [float(x['party']) for x in trainData]

In [33]:
Y_test = [float(x['party']) for y in testData]

In [34]:
h = model.fit(x=X_train, y=np.asarray(Y_train), epochs=100)

Train on 80 samples
Epoch 1/100
80/80 [==============================] - 3s 34ms/sample - loss: 0.6927 - binary_accuracy: 0.5875
Epoch 2/100
80/80 [==============================] - 0s 674us/sample - loss: 0.6921 - binary_accuracy: 0.6125
Epoch 3/100
80/80 [==============================] - 0s 612us/sample - loss: 0.6916 - binary_accuracy: 0.6250
Epoch 4/100
80/80 [==============================] - 0s 601us/sample - loss: 0.6912 - binary_accuracy: 0.6375
Epoch 5/100
80/80 [==============================] - 0s 629us/sample - loss: 0.6907 - binary_accuracy: 0.6375
Epoch 6/100
80/80 [==============================] - 0s 596us/sample - loss: 0.6903 - binary_accuracy: 0.6375
Epoch 7/100
80/80 [==============================] - 0s 1ms/sample - loss: 0.6899 - binary_accuracy: 0.6500
Epoch 8/100
80/80 [==============================] - 0s 1ms/sample - loss: 0.6894 - binary_accuracy: 0.6750
Epoch 9/100
80/80 [==============================] - 0s 1ms/sample - loss: 0.6890 - binary_accuracy: 0.70

This is almost certainly overfit, but time for....

## Evaluating

In [35]:
predictions = model.predict(x=X_test)

In [36]:
score = model.evaluate(X_test, np.asarray(Y_test))

8/8 [==============================] - 1s 68ms/sample - loss: 0.4438 - binary_accuracy: 0.8750


Well, 88% isn't bad. Maybe. Well, that's all I can do here for now. I may revisit this with better data.
-RB